In [325]:
import tweepy
import re
from textblob import TextBlob
import pandas as pd
from pandas import DataFrame



import numpy as np
import joblib
import pickle
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [326]:
api_key = 'FAQUNHC2FtBdXma5JwdbJsCt6'
api_secret_key = 'f8ZyKeNNuHoe6JvpPvYQD9keagttbjARQvaDrAqWfpVW28V4EY'
access_token = '1386283874901127168-2uJkEOsfczHSJB1tt598kBgvJ6ia5v'
access_token_secret = 'GYlCd3SnlTxVVfUWKZSjBGWFGuQ4mqngvMJRZvvu5Mxhg'

In [327]:
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [393]:
hasilSearch = api.search(q="vaksin",lang="en", count=1000)

In [394]:
hasil = []

In [395]:
for tweet in hasilSearch:
    tweet_properties = {}
    tweet_properties['tanggal_tweet'] = tweet.created_at
    tweet_properties['pengguna'] = tweet.user.screen_name
    tweet_properties['isi_tweet'] = tweet.text
    tweet_bersih = " ".join(re.sub("(@[A-Za-z0-9]+)|(^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.text).split())
    
    analysis = TextBlob(tweet_bersih)
    
    if analysis.sentiment.polarity > 0.0:
        tweet_properties["sentimen"] = "positif"
    elif analysis.sentiment.polarity == 0.0:
        tweet_properties["sentimen"] = "netral"
    else:
        tweet_properties["sentimen"] = "negatif"
    
    if tweet.retweet_count > 0:
        if tweet_properties not in hasil:
            hasil.append(tweet_properties)
    else:
        hasil.append(tweet_properties)
    
#     print(tweet_properties)

In [396]:
# hasil

In [397]:
df =  DataFrame(data = hasil)

In [398]:
df.head(5)

,tanggal_tweet,pengguna,isi_tweet,sentimen
0,2021-05-10 15:42:01,pskedelik,RT @amsyr_rosaceae: @inailsins HOW MITI HANDLE...,positif
1,2021-05-10 15:37:54,Alva47831808,RT @astridwulan: THREAD \n\nDapet vaksin Astra...,netral
2,2021-05-10 15:33:19,amrllakml,RT @pojiepooh: How to convince your parents to...,netral
3,2021-05-10 15:24:29,zmaamor,RT @pojiepooh: How to convince your parents to...,netral
4,2021-05-10 15:23:22,Marshmellowjelo,RT @pojiepooh: How to convince your parents to...,netral


In [399]:
x = df['isi_tweet']
y = df['sentimen']

In [400]:
bow_transformer = CountVectorizer()
print(df['isi_tweet'].shape)
x = bow_transformer.fit_transform(df['isi_tweet'])

print(x.toarray())
print('shape of Sparse Matriks: ', x.shape)
print('Amount of Non-Zero occurrences: ', x.nnz)

filename1 = 'count_vectorized2.pkl'
pickle.dump(bow_transformer, open(filename1, 'wb'))

tf_transform = TfidfTransformer(use_idf=False).fit(x)
x = tf_transform.transform(x)
print(x.shape)

filename1 = 'tfid_transform2.pkl'
pickle.dump(tf_transform, open(filename1, 'wb'))

(31,)
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
shape of Sparse Matriks:  (31, 292)
Amount of Non-Zero occurrences:  476
(31, 292)


In [401]:
density = (100.0 * x.nnz / (x.shape[0] * x.shape[1]))
print('Density: {}'.format((density)))

Density: 5.2585064074237735


In [402]:
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [403]:
#Naive Bayes Method 
#classifier data
nb = MultinomialNB()
nb.fit(x_train, y_train)
preds = nb.predict(x_test)

In [404]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00         1
      netral       0.71      1.00      0.83         5
     positif       0.00      0.00      0.00         1

    accuracy                           0.71         7
   macro avg       0.24      0.33      0.28         7
weighted avg       0.51      0.71      0.60         7



c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [405]:
from io import StringIO
classification = classification_report(y_test, preds)
s = StringIO(classification)
with open('classification.csv', 'w') as f:
    for line in s:
        f.write(line)

c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [406]:
print (accuracy_score(y_test, preds))

0.7142857142857143


# **-------------------------------------------------------------------------------------------------**

In [355]:
accuracy = accuracy_score(y_test,preds)
a = np.asarray([accuracy])
np.savetxt('accuracy.csv', a, delimiter=',', fmt='%s')

In [356]:
Final = df[['tanggal_tweet','pengguna','isi_tweet','sentimen']]

In [357]:
Final.to_csv('hasil_analysis.csv')

In [358]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [359]:
filename = 'model_analisis.pkl'
pickle.dump(model, open(filename, 'wb'))

In [360]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.75


In [361]:
loaded_model.predict(x_test)

array(['netral', 'positif', 'positif', 'netral', 'netral', 'netral',
       'netral', 'netral', 'netral', 'positif', 'positif', 'netral',
       'netral', 'positif', 'netral', 'positif', 'netral', 'positif',
       'netral', 'netral'], dtype=object)